In [ ]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import re
from time import sleep
import json
import os

import aiohttp
import asyncio
import pathlib

In [ ]:
category = {'Завтраки':'zavtraki', 
            'Закуски':'zakuski', 'Напитки':'napitki', 
            'Основные блюда':'osnovnye-blyuda', 'Паста и пицца':'pasta-picca', 
            'Салаты':'salaty', 
            'Соусы и маринады':'sousy-marinady', 'Супы':'supy', 
            'Выпечка и десерты':'vypechka-deserty'}


kuhni = {'Итальянская кухня':'italyanskaya-kuhnya', 'Грузинская кухня':'gruzinskaya-kuhnya', 
            'Китайская кухня':'kitayskaya-kuhnya', 'Французская кухня':'francuzskaya-kuhnya', 
            'Русская кухня':'russkaya-kuhnya', 'Японская кухня':'yaponskaya-kuhnya', 
            'Индийская кухня':'indiyskaya-kuhnya', 'Мексиканская кухня':'meksikanskaya-kuhnya', 
            'Армянская кухня':'armyanskaya-kuhnya', 'Американская кухня':'amerikanskaya-kuhnya', 
            'Испанская кухня':'ispanskaya-kuhnya', 'Немецкая кухня':'nemeckaya-kuhnya', 
            'Греческая кухня':'grecheskaya-kuhnya', 'Азербайджанская кухня':'azerbaydzhanskaya-kuhnya', 
        }   
         
         # 'Европейская кухня':'evropeyskaya-kuhnya', 'Еврейская кухня':'evreyskaya-kuhnya', 
         #    'Корейская кухня':'koreyskaya-kuhnya', 'Тайская кухня':'tayskaya-kuhnya', 
         #    'Паназиатская кухня':'panaziatskaya-kuhnya', 'Турецкая кухня':'tureckaya-kuhnya', 
         #    'Узбекская кухня':'uzbekskaya-kuhnya', 'Татарская кухня':'tatarskaya-kuhnya', 
         #    'Средиземноморская кухня':'sredizemnomorskaya-kuhnya', 'Арабская кухня':'arabskaya-kuhnya', 
         #    'Украинская кухня':'ukrainskaya-kuhnya', 'Польская кухня':'polskaya-kuhnya', 
         #    'Британская кухня':'britanskaya-kuhnya', 'Белорусская кухня':'belorusskaya-kuhnya', 
         #    'Норвежская кухня':'norvezhskaya-kuhnya', 'Шведская кухня':'shvedskaya-kuhnya', 
         #    'Марокканская кухня':'marokkanskaya-kuhnya', 'Болгарская кухня':'bolgarskaya-kuhnya', 
         #    'Австрийская кухня':'avstriyskaya-kuhnya', 'Австралийская кухня':'avstraliyskaya-kuhnya', 
         #    'Финская кухня':'finskaya-kuhnya', 'Сербская кухня':'serbskaya-kuhnya', 
         #    'Венгерская кухня':'vengerskaya-kuhnya', 'Вьетнамская кухня':'vetnamskaya-kuhnya', 
         #    'Молдавская кухня':'moldavskaya-kuhnya', 'Осетинская кухня':'osetinskaya-kuhnya', 
         #    'Канадская кухня':'canadian-cuisine', 'Алжирская кухня':'algerian-cuisine', 
         #    'Дагестанская кухня':'dagestan-cuisine', 'Албанская кухня':'albanian-cuisine', 
         #    'Мировая кухня':'world-cuisine', 'Туркменская кухня':'turkmenskaya-kuhnya', 
         #    'Афганская кухня':'afganskaya-kuhnya', 'Египетская кухня':'egipetskaya-kuhnya', 
         #    'Крымская кухня':'krymskaya-kuhnya', 
         #    'Африканская кухня':'afikanskaya-kuhnya', 'Филиппинская кухня':'filippinskaya-kuhnya', 
         #    'Шотландская кухня':'shotlandskaya-kuhnya', 'Сирийская кухня':'siriyskaya-kuhnya', 
         #    'Таджикская кухня':'tadzhikskaya-kuhnya', 'Португальская кухня':'portugalskaya-kuhnya', 
         #    'Латиноамериканская кухня':'latinoamerikanskaya-kuhnya', 'Латвийская кухня':'latviyskaya-kuhnya', 
         #    'Индонезийская кухня':'indoneziyskaya-kuhnya', 'Датская кухня':'datskaya-kuhnya', 
         #    'Бурятская кухня':'buryat-cuisine', 'Кавказская кухня':'caucasus-cuisine', 
         #    'Колумбийская кухня':'gastronomia-de-colombia', 'Креольская кухня':'louisiana-cuisine', 
         #    'Сицилийская кухня':'sicilian-cuisine', 'Советская кухня':'soviet-cuisine', 
         #    'Камбоджийская кухня':'kambodzhiyskaya-kuhnya', 
         #    'Казахская кухня':'kazahskaya-kuhnya', 'Чеченская кухня':'chechenskaya-kuhnya', 
         #    'Южно-индийская кухня':'yuzhno-indiyskaya-kuhnya', 
         #    'Скандинавская кухня':'skandinavskaya-kuhnya', 'Бразильская кухня':'brazilskaya-kuhnya', 
         #    'Кухня Вестероса':'vesterosskaya-kuhnya', 
         #    'Голландская кухня':'gollandskaya-kuhnya', 'Чешская кухня':'cheshskaya-kuhnya', 
         #    'Литовская кухня':'litovskaya-kuhnya', 'Ливанская кухня':'livanskaya-kuhnya', 
         #    'Кубинская кухня':'kubinskaya-kuhnya', 'Киргизская кухня':'kirgizskaya-kuhnya', 
         #    'Башкирская кухня':'bashkirskaya-kuhnya', 'Исландская кухня':'icelandic-cuisine', 
         #    'Швейцарская кухня':'swiss-cuisine', 'Абхазская кухня':'abkhaziancuisine', 
         #    'Хорватская кухня':'khorvatskaya-kuhnya', 
         #    'Румынская кухня':'rumynskaya-kuhnya', 'Персидская кухня':'persidskaya-kuhnya', 
         #    'Сингапурская кухня':'singapurskaya-kuhnya', 'Черкесская кухня':'cherkesskaya-kuhnya', 
         #    'Восточно-индийская кухня':'vostochno-indiyskaya-kuhnya', 'Одесская кухня':'odesskaya-kuhnya', 
         #    'Малайзийская кухня':'malaziyskaya-kuhnya', 'Ирландская кухня':'irlandskaya-kuhnya', 
         #    'Авторская кухня':'avtorskaya-kuhnya', 'Югославская кухня':'yugoslavskaya-kuhnya', 
         #    'Эстонская кухня':'estonskaya-kuhnya', 'Карибская кухня':'karibskaya-kuhnya', 
         #    'Бельгийская кухня':'belgiyskaya-kuhnya', 'Аргентинская кухня':'argentinskaya-kuhnya'}

menu = {'Безглютеновая диета':'bezglyutenovaya-dieta', 'Вегетарианская еда':'vegetarianskaya-eda', 
            'Веганская еда':'veganskaya-eda', 'Безлактозная диета':'bezlaktoznaya-dieta', 
            'Детское меню':'detskoe-menyu', 'Низкокалорийная еда':'nizkokaloriynaya-eda', 
            'Постная еда':'postnaya-eda', 'Меню при диабете':'menu-pri-diabete'}

In [ ]:
# #Не обязательно запускать
# S = 0
# for el_kuhni in kuhni:
#     for el_category in category:
#         print(el_kuhni, kuhni[el_kuhni])
#         print(el_category, category[el_category])
#         url  = 'https://eda.ru/recepty/' + kuhni[el_kuhni] + '/' + category[el_category]
#         r = requests.get(url)
#         break;
#         soup = BeautifulSoup(r.text)
#         # # print(soup)
#         # tree = html.fromstring(soup)
#         num_recipes = soup.find('span', {'class': 'emotion-1ad0u8b'}).text 
#     #     print(num_recipes)
#         print(re.search('\d+', num_recipes)[0])
#         if int(re.search('\d+', num_recipes)[0]) >= 10000:
#             print('!!!')
#         S += int(re.search('\d+', num_recipes)[0])
#     break;
# S

In [ ]:
# async with aiohttp.ClientSession() as session:
#     async with session.get('http://httpbin.org/get') as resp:
#         # print(resp.status)
#         print(await resp.text())


# async def async_generator():
#     for i in range(10):
#         # block to simulate doing work
#         await asyncio.sleep(1)
#         # yield the result
#         yield i

# async for x in async_generator():
#     print(x)

In [ ]:
# url  = 'https://eda.ru/recepty/vypechka-deserty/brauni-brownie-20955'
def get_data(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    # Тут находим теги
    recipe_tags = soup.find('ul', {'class': 'emotion-1bqzczv'})
    menu_tag = 'Еда без ограничений'
    for child in recipe_tags.children:
#         print(child.text)
        if child.text in menu:
            menu_tag = child.text
    # Тут находим название блюда
    recipe_title = soup.find('h1', {'class': 'emotion-gl52ge'})
    title = recipe_title.text 
#     print(title)
    # Тут находим описание блюда
    recipe_descr = soup.find('span', {'class': 'emotion-1x1q7i2'})
    descripion = recipe_descr.text if recipe_descr is not None else ''
#     print(descripion)
    return {'title': title, 'menu': menu_tag, 'description': descripion}


In [ ]:
# url  = 'https://eda.ru/recepty/vypechka-deserty/brauni-brownie-20955'
# get_data(url)

In [ ]:
# url  = 'https://eda.ru/recepty?page=714'
def get_sites(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    
    all_sites = []
    
    list_recipes = soup.find('div', {'class': 'emotion-7ote3x'})
    for child in list_recipes.children:
        for recipe in child.children:
            if recipe.find('a', {'class': 'emotion-12sjte8'}) is not None:
#                 print(recipe.find('a', {'class': 'emotion-12sjte8'}).get('href'), '\n\n\n')
                all_sites.append('https://eda.ru' + recipe.find('a', {'class': 'emotion-12sjte8'}).get('href'))

    list_recipes = soup.find('div', {'class': 'emotion-1ywwzp6'})
    for child in list_recipes.children:
        for recipe in child.children:
            if recipe.find('a', {'class': 'emotion-12sjte8'}) is not None:
#                 print(recipe.find('a', {'class': 'emotion-12sjte8'}).get('href'), '\n\n\n')
                all_sites.append('https://eda.ru' + recipe.find('a', {'class': 'emotion-12sjte8'}).get('href'))
    return all_sites


In [ ]:
# url  = 'https://eda.ru/recepty?page=714'
# get_sites(url)

In [ ]:
el_category = 'Выпечка и десерты'

category_path = category[el_category]
pathlib.Path(category_path).mkdir(exist_ok=True)

status_path = 'find_markers'
pathlib.Path(status_path).mkdir(exist_ok=True)

for el_kuhni in kuhni:
#     for el_category in category:
    print(el_kuhni, kuhni[el_kuhni])
    print('----')
#     print(el_category, category[el_category])
    url  = 'https://eda.ru/recepty/' + kuhni[el_kuhni] + '/' + category[el_category]
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    num_recipes = soup.find('span', {'class': 'emotion-1ad0u8b'}).text
    num_recipes = int(re.search('\d+', num_recipes)[0])

    on_one_page = 14
    num_pages = (num_recipes - 1)//on_one_page + 1
    max_page = min(714, num_pages)
    
    
    status_filename = os.path.join(status_path, f'we_are_at_page_number_{kuhni[el_kuhni]}.txt')
    try:
        with open(status_filename, 'r') as f:
            start_page = int(f.readline()) + 1
    except:
        start_page = 1
        
    for i in range(start_page, max_page+1):
        url_in_loop = url + '?page=' + str(i)
        sites = get_sites(url_in_loop)
        for dish_url in sites:
            print(dish_url)
            
            # now we're going to create filename to write to:
            fname = os.path.join(category[el_category], dish_url.split('/')[-1] + '.json')
            
            if not os.path.exists(fname):
                try:
                    data = get_data(dish_url)
                except:
                    with open("error_url.txt", "a+", encoding="utf-8") as write_file:
                        write_file.write(dish_url)
                        write_file.write("\n")
                        continue
                print(data)
                data_total = data | {'category': el_category, 'cuisine': el_kuhni}
                with open(fname, 'w', encoding='utf-8') as write_file:
                    json.dump(data_total, write_file, ensure_ascii=False)
                    write_file.write("\n")

                sleep(1.5)
            else:
                print(f'already were at "{fname}"')
        with open(status_filename, 'w') as f:
            f.write(str(i))
        print('-------')
        print(f"we're done with page {i}!")
        print('')
        sleep(2)
#         break


In [ ]:
714*14

In [ ]:
# with open("data_file.json", "r+", encoding='utf-8') as write_file:
#     counter = 0
#     for line in write_file:
#         data_ = json.loads(line)
# #         if not (data_['category'] == 'Завтраки' and data_['cuisine'] == 'Итальянская кухня'):
# #         print(data_['category'], data_['cuisine'])
#         counter += 1
#     print(counter)           